In [ ]:
# !pip install datasets torch transformers evaluate Tiktoken accelerate==0.26.0

  Using cached dill-0.3.8-py3-none-any.whl.metadata (10 kB)
  Using cached fsspec-2024.12.0-py3-none-any.whl.metadata (11 kB)
Using cached dill-0.3.8-py3-none-any.whl (116 kB)
Using cached fsspec-2024.12.0-py3-none-any.whl (183 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 4.6 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [1]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import evaluate
from datasets import load_dataset

In [2]:
dataset = load_dataset("AI4Math/MathVista")

In [3]:
test_data = dataset["testmini"]

In [8]:
import torch
from transformers import AutoModelForVision2Seq, AutoTokenizer, BlipProcessor, pipeline

import torch
from transformers import AutoModelForCausalLM, AutoModelForSequenceClassification, AutoModelForSeq2SeqLM, AutoTokenizer, pipeline

# Check if MPS (Apple Silicon) is being used
device = "mps" if torch.backends.mps.is_available() else "cuda" if torch.cuda.is_available() else "cpu"

from transformers import AutoModelForCausalLM, AutoModelForSequenceClassification, AutoModelForSeq2SeqLM, AutoModelForMaskedLM, AutoTokenizer

def load_model(model_name):
    """Loads the correct model type based on architecture."""
    
    if "bert" in model_name:  # For BERT models
        model = AutoModelForSequenceClassification.from_pretrained(model_name)
    
    elif "t5" in model_name or "flan" in model_name:  # For Flan-T5 or T5 models
        model = AutoModelForSeq2SeqLM.from_pretrained(
            model_name,
            device_map="auto",
            torch_dtype=torch.float32 if device == "mps" else torch.float16,  # MPS fix
            offload_folder="offload_weights"
        )
    
    elif "longformer" in model_name:  # For Longformer models
        model = AutoModelForMaskedLM.from_pretrained(
            model_name,
            device_map="auto",
            torch_dtype=torch.float32 if device == "mps" else torch.float16,
            offload_folder="offload_weights"
        )

    else:  # For causal LMs (GPT, OPT, Falcon)
        model = AutoModelForCausalLM.from_pretrained(
            model_name,
            device_map="auto",
            torch_dtype=torch.float32 if device == "mps" else torch.float16,
            offload_folder="offload_weights"
        )
    
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    return model, tokenizer


def evaluate_model(model_name, dataset, max_samples=100):
    """Evaluates a model using accuracy on the MathVista dataset."""
    
    # Load tokenizer and model
    if "blip" in model_name:
        processor = BlipProcessor.from_pretrained(model_name)
        model = AutoModelForVision2Seq.from_pretrained(model_name, device_map="auto")
        pipe = pipeline("image-to-text", model=model, tokenizer=processor.tokenizer)
    else:
        model, tokenizer = load_model(model_name)
        pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)

    correct, total = 0, 0
    for sample in dataset.select(range(min(max_samples, len(dataset)))):
        question = sample["question"]
        image = sample["decoded_image"]  # Extract image
        answer = sample["answer"]

        # Ensure valid input
        if "blip" in model_name and image is not None:
            response = pipe(image)[0]["generated_text"]  # Get caption
        else:
            response = pipe(
                question,
                max_new_tokens=50,  
                pad_token_id=tokenizer.pad_token_id,
                truncation=True,
                do_sample=False
            )[0]["generated_text"]

        # Check if answer is correct
        if answer.strip().lower() in response.strip().lower():
            correct += 1
        total += 1

    # Compute accuracy
    acc = correct / total if total > 0 else 0
    return {"model": model_name, "accuracy": acc}

In [82]:
print(evaluate_model("google/flan-t5-small", test_data, max_samples=100))

Device set to use mps


{'model': 'google/flan-t5-small', 'accuracy': 0.16}


In [5]:
import torch
from transformers import BlipProcessor, BlipForQuestionAnswering

def evaluate_blip_vqa(dataset, max_samples=100):
    """Evaluates BLIP VQA model on MathVista dataset."""
    
    model_name = "Salesforce/blip-vqa-base"
    processor = BlipProcessor.from_pretrained(model_name)
    model = BlipForQuestionAnswering.from_pretrained(model_name).to("mps" if torch.backends.mps.is_available() else "cuda")

    correct, total = 0, 0
    for sample in dataset.select(range(min(max_samples, len(dataset)))):
        question = sample["question"]
        image = sample["decoded_image"]

        # Ensure image is valid
        if image is None:
            continue  

        # Process image + question
        inputs = processor(images=image, text=question, return_tensors="pt").to(model.device)
        outputs = model.generate(**inputs)
        response = processor.batch_decode(outputs, skip_special_tokens=True)[0]

        # Compare with expected answer
        answer = sample["answer"]
        if answer and answer.strip().lower() in response.strip().lower():
            correct += 1
        total += 1

    acc = correct / total if total > 0 else 0
    return {"model": model_name, "accuracy": acc}

# Run BLIP VQA Model
print(evaluate_blip_vqa(test_data, max_samples=100))

{'model': 'Salesforce/blip-vqa-base', 'accuracy': 0.13}


In [8]:
import torch
from transformers import ViltProcessor, ViltForQuestionAnswering
from PIL import Image

def evaluate_vilt_vqa_light(dataset, max_samples=100):
    """Evaluates ViLT VQA Base on MathVista dataset."""

    model_name = "dandelin/vilt-b32-finetuned-vqa"
    processor = ViltProcessor.from_pretrained(model_name)
    model = ViltForQuestionAnswering.from_pretrained(model_name).to(
        "mps" if torch.backends.mps.is_available() else "cuda"
    )

    correct, total = 0, 0
    for sample in dataset.select(range(min(max_samples, len(dataset)))):
        question = sample["question"]
        image = sample["decoded_image"]

        if image is None:
            continue  

        # Convert image to RGB format (fix channel dimension issue)
        if isinstance(image, Image.Image):  
            image = image.convert("RGB")

        # Process image + question with proper truncation
        inputs = processor(
            images=image,
            text=question,
            return_tensors="pt",
            truncation=True,   # Fix token length issue
            max_length=40,     # Set max sequence length to avoid overflow
            padding="max_length"
        ).to(model.device)

        outputs = model(**inputs)
        predicted_answer = processor.tokenizer.decode(outputs.logits.argmax(-1), skip_special_tokens=True)

        answer = sample["answer"]
        if answer and answer.strip().lower() in predicted_answer.strip().lower():
            correct += 1
        total += 1

    acc = correct / total if total > 0 else 0
    return {"model": model_name, "accuracy": acc}

# Run ViLT VQA Model
print(evaluate_vilt_vqa_light(test_data, max_samples=100))

{'model': 'dandelin/vilt-b32-finetuned-vqa', 'accuracy': 0.06}


In [16]:
!pip install transformers openxlab

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 953.1/953.1 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 11.4 MB/s eta 0:00:00a 0:00:01
  Created wheel for oss2: filename=oss2-2.17.0-py3-none-any.whl size=112388 sha256=a0bf4783fef774df0920df5deeb62eb2c35f524f7b8bb96f16f151c1854beee6
  Stored in directory: /Users/swatisharma/Library/Caches/pip/wheels/87/04/7b/7e61b8157fdf211c5131375240d0d86ca82e2a88ead9672c88
  Created wheel for aliyun-python-sdk-core: filename=aliyun_python_sdk_core-2.16.0-py3-none-any.whl size=535320 sha256=0c624f7b740a31e0da5e2c851120f9c997bc2c89246a62b8b2133823cd2b2b54
  Stored in directory: /Users/swatisharma/Library/Caches/pip/wheels/35/11/5e/08e7cb4e03a3e83b4862edd12d1143c8d3936a3dd57a3ee46d
  Created wheel for crcmod: filename=crcmod-1.

In [ ]:
import torch
from transformers import Blip2Processor, Blip2ForConditionalGeneration

def evaluate_blip2_vqa(dataset, max_samples=100):
    """Evaluates BLIP-2 on MathVista dataset."""

    model_name = "Salesforce/blip2-flan-t5-xl"
    processor = Blip2Processor.from_pretrained(model_name)
    model = Blip2ForConditionalGeneration.from_pretrained(model_name, device_map="auto").to(
        "mps" if torch.backends.mps.is_available() else "cuda"
    )

    correct, total = 0, 0
    for sample in dataset.select(range(min(max_samples, len(dataset)))):
        question = sample["question"]
        image = sample["decoded_image"]

        if image is None:
            continue  

        # Process image + question
        inputs = processor(images=image, text=question, return_tensors="pt").to(model.device)
        outputs = model.generate(**inputs, max_new_tokens=50)
        response = processor.tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]

        # Compare with expected answer
        answer = sample["answer"]
        if answer and answer.strip().lower() in response.strip().lower():
            correct += 1
        total += 1

    acc = correct / total if total > 0 else 0
    return {"model": model_name, "accuracy": acc}

# Run BLIP-2 VQA Model
print(evaluate_blip2_vqa(test_data, max_samples=100))

In [13]:
import torch
from transformers import InstructBlipProcessor, InstructBlipForConditionalGeneration

def evaluate_instructblip_vqa(dataset, max_samples=100):
    """Evaluates InstructBLIP VQA model on the MathVista dataset."""

    model_name = "Salesforce/instructblip-flan-t5-xl"
    processor = InstructBlipProcessor.from_pretrained(model_name)
    model = InstructBlipForConditionalGeneration.from_pretrained(model_name).to(
        "mps" if torch.backends.mps.is_available() else "cuda"
    )

    correct, total = 0, 0
    for sample in dataset.select(range(min(max_samples, len(dataset)))):
        question = sample["question"]
        image = sample["decoded_image"]

        if image is None:
            continue

        inputs = processor(images=image, text=question, return_tensors="pt").to(model.device)
        outputs = model.generate(**inputs)
        predicted_answer = processor.decode(outputs[0], skip_special_tokens=True)

        answer = sample["answer"]
        if answer and answer.strip().lower() in predicted_answer.strip().lower():
            correct += 1
        total += 1

    acc = correct / total if total > 0 else 0
    return {"model": model_name, "accuracy": acc}

# Run InstructBLIP VQA Model
print(evaluate_instructblip_vqa(test_data, max_samples=100))

preprocessor_config.json:   0%|          | 0.00/439 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/21.2k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/43.0 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/833 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/2.31k [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/135k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

KeyboardInterrupt: 

In [17]:
import torch
from transformers import QwenImageProcessor, QwenForVisionLanguageUnderstanding

def evaluate_qwen_vl(dataset, max_samples=100):
    """Evaluates Qwen-VL model on the MathVista dataset."""

    model_name = "Qwen/Qwen-VL"
    processor = QwenImageProcessor.from_pretrained(model_name)
    model = QwenForVisionLanguageUnderstanding.from_pretrained(model_name).to(
        "mps" if torch.backends.mps.is_available() else "cuda"
    )

    correct, total = 0, 0
    for sample in dataset.select(range(min(max_samples, len(dataset)))):
        question = sample["question"]
        image = sample["decoded_image"]

        if image is None:
            continue

        inputs = processor(images=image, text=question, return_tensors="pt").to(model.device)
        outputs = model(**inputs)
        predicted_answer = processor.decode(outputs.logits.argmax(-1), skip_special_tokens=True)

        answer = sample["answer"]
        if answer and answer.strip().lower() in predicted_answer.strip().lower():
            correct += 1
        total += 1

    acc = correct / total if total > 0 else 0
    return {"model": model_name, "accuracy": acc}

# Run Qwen-VL Model
print(evaluate_qwen_vl(test_data, max_samples=100))

ImportError: cannot import name 'QwenImageProcessor' from 'transformers' (/Users/swatisharma/.pyenv/versions/3.10.0/lib/python3.10/site-packages/transformers/__init__.py)